# Create a simple tensor with random items

In [1]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate 20 parameters randomly distributed between -50 and +150
params = np.random.rand(20) * 100 - 50

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[ 44.12 -47.35   0.   -10.59  24.84 -28.74 -31.53 -36.    43.12   7.41
  13.89  31.85 -20.58 -46.35 -16.54  -3.2   -7.46   5.06  -0.54  26.58]


# Define the quantization methods and quantize

In [2]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Also known as symmetric quantization
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Asymmetric scale: 0.35870588235294115, zero: 132.0
[255   0 132 102 201  52  44  32 252 153 171 221  75   3  86 123 111 146
 130 206]

Symmetric scale: 0.37283464566929136
[ 118 -127    0  -28   67  -77  -85  -97  116   20   37   85  -55 -124
  -44   -9  -20   14   -1   71]


In [3]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[ 44.12 -47.35   0.   -10.59  24.84 -28.74 -31.53 -36.    43.12   7.41
  13.89  31.85 -20.58 -46.35 -16.54  -3.2   -7.46   5.06  -0.54  26.58]

Dequantize Asymmetric:
[ 44.12 -47.35   0.   -10.76  24.75 -28.7  -31.57 -35.87  43.04   7.53
  13.99  31.92 -20.45 -46.27 -16.5   -3.23  -7.53   5.02  -0.72  26.54]

Dequantize Symmetric:
[ 43.99 -47.35   0.   -10.44  24.98 -28.71 -31.69 -36.16  43.25   7.46
  13.79  31.69 -20.51 -46.23 -16.4   -3.36  -7.46   5.22  -0.37  26.47]


In [4]:
# Calculate the quantization error
print(f'Asymmetric error: {quantization_error(params, params_deq_asymmetric)}')
print(f'Symmetric error: {quantization_error(params, params_deq_symmetric)}')

Asymmetric error: 0.007958602076124584
Symmetric error: 0.014627939115878081
